In [1]:
import pandas as pd
import numpy as np
import os
import math
from scipy import signal
import scipy.io
from scipy import signal
import mne
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
import tensorflow
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.layers import Dense, Activation, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Flatten, Dropout, BatchNormalization, Input,UpSampling1D
from tensorflow.keras.layers import concatenate, Lambda, Conv2D, MaxPooling2D, GlobalAveragePooling2D,LSTM,Activation
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import shuffle

In [2]:
trainctr = 0
X_train = np.zeros((1300,31,500))
Y_train = np.zeros((1300,))


for fi in os.listdir('Segmented/'):
    eeg_sig = np.load(os.path.join('Segmented/',fi))
    eeg_sig_var = eeg_sig[250:750,:]
    _,char,trial = fi.split('_')
    trial,_ = trial.split('.')
    
    for i in range(0,31):
        X_train[trainctr,i,:] = np.transpose(eeg_sig_var[:,i])
    Y_train[trainctr] = ord(char)-65
    trainctr = trainctr+1
        
print(trainctr)

1300


In [3]:
X_train_final, X_test_final, Y_train_final, Y_test_final = train_test_split(X_train, Y_train, test_size=0.2, random_state=1)

In [4]:
for i in range(0,len(X_train_final)):
    X_train_final[i,:,:] = (X_train_final[i,:,:]-np.mean(X_train_final[i,:,:]))/np.std(X_train_final[i,:,:])

for i in range(0,len(X_test_final)):
    X_test_final[i,:,:] = (X_test_final[i,:,:]-np.mean(X_test_final[i,:,:]))/np.std(X_test_final[i,:,:])

In [5]:
tensorflow.keras.backend.clear_session()
from EEGModels import EEGNet, ShallowConvNet, DeepConvNet
model = EEGNet(nb_classes=26,Chans=31,Samples=500)
print(model.summary())
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_accuracy', verbose=1, patience=20,restore_best_weights=True)
model.fit(X_train_final, y=to_categorical(Y_train_final),validation_split=0.2,epochs=500, batch_size=8,verbose=1,callbacks=[es])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 31, 500, 1)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 31, 500, 8)        512       
_________________________________________________________________
batch_normalization (BatchNo (None, 31, 500, 8)        32        
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 1, 500, 16)        496       
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 500, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 1, 500, 16)        0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 1, 125, 16)        0     

104/104 [==============================] - 1s 10ms/step - loss: 2.4501 - accuracy: 0.3101 - val_loss: 2.9477 - val_accuracy: 0.1538
Epoch 41/500
104/104 [==============================] - 1s 10ms/step - loss: 2.4275 - accuracy: 0.3029 - val_loss: 2.9461 - val_accuracy: 0.1490
Epoch 42/500
104/104 [==============================] - 1s 10ms/step - loss: 2.4219 - accuracy: 0.3377 - val_loss: 2.9889 - val_accuracy: 0.1298
Epoch 43/500
104/104 [==============================] - 1s 9ms/step - loss: 2.4288 - accuracy: 0.3197 - val_loss: 2.9762 - val_accuracy: 0.1346
Epoch 44/500
104/104 [==============================] - 1s 9ms/step - loss: 2.3908 - accuracy: 0.3137 - val_loss: 2.9701 - val_accuracy: 0.1442
Restoring model weights from the end of the best epoch.
Epoch 00044: early stopping


In [6]:
pred = model.predict(X_test_final)
Y_pred = np.argmax(pred,axis=1)
print(accuracy_score(Y_pred,Y_test_final)*100)

17.692307692307693
